In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e5/sample_submission.csv
/kaggle/input/playground-series-s4e5/train.csv
/kaggle/input/playground-series-s4e5/test.csv


This is a test to see if commits work.

In [2]:
# Importing libraries
import matplotlib.pyplot as plt
import seaborn as sns

from catboost import CatBoostRegressor

from sklearn.preprocessing import StandardScaler, PolynomialFeatures, OneHotEncoder,StandardScaler,PowerTransformer, MinMaxScaler, RobustScaler
from sklearn.model_selection import KFold, cross_val_predict, train_test_split,GridSearchCV,cross_val_score
from sklearn.linear_model import LinearRegression, Lasso, Ridge,ElasticNet
from sklearn.metrics import r2_score,mean_squared_error
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
from sklearn.impute import SimpleImputer

from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR


from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.feature_selection import SelectKBest,SelectPercentile,f_classif,f_regression,mutual_info_regression,mutual_info_classif,SelectFromModel,RFE

from xgboost import XGBRegressor
from xgboost import plot_importance

import optuna

#importing plotly and cufflinks in offline mode
import cufflinks as cf
import plotly.offline
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

import warnings
warnings.filterwarnings('ignore')

In [3]:
url_train = '/kaggle/input/playground-series-s4e5/train.csv'
url_test = '/kaggle/input/playground-series-s4e5/test.csv'
url_sub = '/kaggle/input/playground-series-s4e5/sample_submission.csv'

df_train = pd.read_csv(url_train).drop('id', axis=1)
df_test = pd.read_csv(url_test).drop('id', axis=1)
df_sub = pd.read_csv(url_sub)

In [5]:
scaler = MinMaxScaler()

df_train[df_train.columns[:-1]] = scaler.fit_transform(df_train[df_train.columns[:-1]])
df_test[df_test.columns] = scaler.transform(df_test)

In [4]:
X = df_train.drop('FloodProbability', axis=1)
y = df_train['FloodProbability']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=41)

In [33]:
test_df

0          0.445
1          0.450
2          0.530
3          0.535
4          0.415
           ...  
1117952    0.495
1117953    0.480
1117954    0.485
1117955    0.495
1117956    0.560
Name: FloodProbability, Length: 1117957, dtype: float64

In [7]:
features = list(df_train.drop(columns=['FloodProbability']).columns)

test_df = df_train['FloodProbability']
train_df = df_train.drop('FloodProbability', axis=1)

def add_features(df):
    df['add_features'] = df[features].sum(axis=1)
    df['multiply_features'] = df[features].prod(axis=1)
#     df['avg_features'] = df[features].mean(axis=1)
    return df

train_df = add_features(train_df)
df_test = add_features(df_test)
train_df

,MonsoonIntensity,TopographyDrainage,RiverManagement,Deforestation,Urbanization,ClimateChange,DamsQuality,Siltation,AgriculturalPractices,Encroachments,...,CoastalVulnerability,Landslides,Watersheds,DeterioratingInfrastructure,PopulationScore,WetlandLoss,InadequatePlanning,PoliticalFactors,add_features,multiply_features
0,5,8,5,8,6,4,4,3,3,4,...,3,3,5,4,7,5,7,3,94,7315660800000
1,6,7,4,4,8,8,3,5,4,6,...,2,0,3,5,3,3,4,3,94,0
2,6,5,6,7,3,7,1,5,4,5,...,3,7,5,6,8,2,3,3,99,10081895040000
3,3,4,6,5,4,8,4,7,6,8,...,4,7,4,4,6,5,7,5,104,72831467520000
4,5,3,2,6,4,4,3,3,3,3,...,2,6,6,4,1,2,3,5,72,20155392000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1117952,3,3,4,10,4,5,5,7,10,4,...,8,7,2,2,1,4,6,4,99,4551966720000
1117953,2,2,4,3,9,5,8,1,3,5,...,4,4,3,7,4,9,4,5,96,2821754880000
1117954,7,3,9,4,6,5,9,1,3,4,...,5,5,5,6,5,5,2,4,98,9185400000000
1117955,7,3,3,7,5,2,3,4,6,4,...,8,5,3,4,6,7,6,4,99,22122558259200


In [35]:
model = LinearRegression()

model.fit(X_train, y_train)
y_pred = model.predict(X_test)
y_pred

array([0.46614334, 0.46626727, 0.53378942, ..., 0.50545244, 0.51131607,
       0.53402991])

In [36]:
r2_score(y_test, y_pred)

0.8452652226576933

In [8]:
y_eval = model.predict(df_test)
y_eval

array([0.57358119, 0.45523396, 0.4548009 , ..., 0.62432794, 0.55090026,
       0.5115132 ])

In [ ]:
model = XGBRegressor(objective='reg:squarederror', 
                         n_estimators=1500, 
                         colsample_bytree=0.8,
                         eta=0.01,
                         max_depth=6,
                         min_child_weight=5,
                         subsample=0.8,
                         device='cuda',
                         seed=42)

model.fit(train_df, test_df)

X_test = add_features(X_test)
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print("Mean Squared Error:", mse)
print("R-squared Score:", r2)

In [ ]:
y_eval = model.predict(df_test)
y_eval

In [ ]:
param_grid = {
    'depth': [10, 12]
    #'l2_leaf_reg': [1, 3, 5, 7, 9]
    # Add other parameters you want to tune
}

# Create CatBoost regressor
model = CatBoostRegressor()

# Perform grid search with cross-validation
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

# Get the best parameters
best_params = grid_search.best_params_
print("Best parameters:", best_params)

# Train the final model with the best parameters
best_model = CatBoostRegressor(**best_params)
best_model.fit(X_train, y_train)

# Make predictions on the test dataset
y_pred = best_model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print("Mean Squared Error:", mse)
print("R-squared Score:", r2)

Learning rate set to 0.113252
0:	learn: 0.0496247	total: 225ms	remaining: 3m 44s
1:	learn: 0.0482811	total: 438ms	remaining: 3m 38s
2:	learn: 0.0470645	total: 652ms	remaining: 3m 36s
3:	learn: 0.0458754	total: 937ms	remaining: 3m 53s
4:	learn: 0.0448330	total: 1.26s	remaining: 4m 10s
5:	learn: 0.0438089	total: 1.55s	remaining: 4m 16s
6:	learn: 0.0428721	total: 1.85s	remaining: 4m 21s
7:	learn: 0.0419662	total: 2.14s	remaining: 4m 25s
8:	learn: 0.0411135	total: 2.43s	remaining: 4m 27s
9:	learn: 0.0402652	total: 2.72s	remaining: 4m 29s
10:	learn: 0.0394895	total: 3.02s	remaining: 4m 31s
11:	learn: 0.0387327	total: 3.32s	remaining: 4m 33s
12:	learn: 0.0380360	total: 3.61s	remaining: 4m 34s
13:	learn: 0.0373762	total: 3.9s	remaining: 4m 34s
14:	learn: 0.0367296	total: 4.2s	remaining: 4m 35s
15:	learn: 0.0361065	total: 4.5s	remaining: 4m 36s
16:	learn: 0.0355298	total: 4.8s	remaining: 4m 37s
17:	learn: 0.0349560	total: 5.1s	remaining: 4m 38s
18:	learn: 0.0344379	total: 5.4s	remaining: 4m 38

In [34]:
model = SVR(kernel='rbf', C=100, gamma='auto')  # You can adjust the hyperparameters C and gamma

# Train the model
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print("Mean Squared Error:", mse)
print("R-squared Score:", r2)

Mean Squared Error: 0.0009220052193961978
R-squared Score: 0.6468556716878666


In [18]:
test_df = df_train['FloodProbability']

model = CatBoostRegressor(iterations=2500, depth=, learning_rate=0.1, loss_function='RMSE', random_seed=42)
model.fit(train_df, test_df, verbose=200)

X_test = add_features(X_test)
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print("Mean Squared Error:", mse)
print("R-squared Score:", r2)

0:	learn: 0.0468408	total: 138ms	remaining: 5m 45s
200:	learn: 0.0185744	total: 21.9s	remaining: 4m 10s
400:	learn: 0.0185153	total: 44.4s	remaining: 3m 52s
600:	learn: 0.0184806	total: 1m 7s	remaining: 3m 33s
800:	learn: 0.0184523	total: 1m 29s	remaining: 3m 10s
1000:	learn: 0.0184260	total: 1m 52s	remaining: 2m 48s
1200:	learn: 0.0184014	total: 2m 16s	remaining: 2m 27s
1400:	learn: 0.0183777	total: 2m 39s	remaining: 2m 4s
1600:	learn: 0.0183559	total: 3m 2s	remaining: 1m 42s
1800:	learn: 0.0183350	total: 3m 26s	remaining: 1m 20s
2000:	learn: 0.0183145	total: 3m 50s	remaining: 57.4s
2200:	learn: 0.0182942	total: 4m 13s	remaining: 34.5s
2400:	learn: 0.0182753	total: 4m 37s	remaining: 11.5s
2499:	learn: 0.0182660	total: 4m 49s	remaining: 0us
Mean Squared Error: 0.0003334081364850402
R-squared Score: 0.8722987788616686


In [19]:
y_eval = model.predict(df_test)
y_eval

array([0.57764096, 0.45351185, 0.44756731, ..., 0.62067617, 0.54837317,
       0.52793586])

In [20]:
df_sub['FloodProbability'] = y_eval
df_sub

,id,FloodProbability
0,1117957,0.577641
1,1117958,0.453512
2,1117959,0.447567
3,1117960,0.466498
4,1117961,0.466265
...,...,...
745300,1863257,0.476703
745301,1863258,0.444384
745302,1863259,0.620676
745303,1863260,0.548373


In [21]:
df_sub.set_index('id').to_csv('df_subCatBoostReg5.csv', header=True, sep=',')

In [6]:
# Detecting columns that have high skewness for the Ridge, Lasso, ElasticNet models
skew_limit = 0.75 # This is our threshold-limit to evaluate skewness. Overall below abs(1) seems acceptable for the linear models. 
skew_vals = df_train.skew()
skew_cols = skew_vals[abs(skew_vals) > skew_limit].sort_values(ascending=False)
skew_cols

rmse_test = []
r2_test = []
model_names = []

s = StandardScaler()
p = PowerTransformer(method='yeo-johnson', standardize=True)

rr = Ridge()
las = Lasso()
el = ElasticNet()
knn = KNeighborsRegressor()

models = [rr,las,el]

for model in models:
    ct = make_column_transformer((s,X.columns),(p,skew_cols.index),remainder='passthrough')  
    pipe = make_pipeline(ct, model)
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)
    rmse_test.append(round(np.sqrt(mean_squared_error(y_test, y_pred)),2))
    r2_test.append(round(r2_score(y_test, y_pred),2))
    print (f'model : {model} and  rmse score is : {round(np.sqrt(mean_squared_error(y_test, y_pred)),2)}, r2 score is {round(r2_score(y_test, y_pred),2)}')

model_names = ['Ridge','Lasso','ElasticNet']
result_df = pd.DataFrame({'RMSE':rmse_test,'R2_Test':r2_test}, index=model_names)
result_df

model : Ridge() and  rmse score is : 0.05, r2 score is 0.85
model : Lasso() and  rmse score is : 0.12, r2 score is -0.0
model : ElasticNet() and  rmse score is : 0.12, r2 score is -0.0


,RMSE,R2_Test
Ridge,0.05,0.85
Lasso,0.12,-0.00
ElasticNet,0.12,-0.00


In [10]:
model = KNeighborsRegressor()

ct = make_column_transformer((s,X.columns),(p,skew_cols.index),remainder='passthrough')  
pipe = make_pipeline(ct, model)
pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)
r2=r2_score(y_test, y_pred)
r2

0.6687673783624568

In [ ]:
rmse_test = []
r2_test = []
model_names = []

rf = RandomForestRegressor(random_state=42)
gb = GradientBoostingRegressor(random_state=42)
et = ExtraTreesRegressor(random_state=42)
xgb = XGBRegressor(random_state=42)

models = [rf, gb, et, xgb]

for model in models:
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    rmse_test.append(round(np.sqrt(mean_squared_error(y_test, y_pred)),4))
    r2_test.append(round(r2_score(y_test, y_pred),4))
    print (f'model: {model} rmse score is : {round(np.sqrt(mean_squared_error(y_test, y_pred)),4)}, r2 score is {round(r2_score(y_test, y_pred),4)}')
    
model_names = ['RandomForest','GradientBoosting','ExtraTrees','XGB']
result_df = pd.DataFrame({'RMSE':rmse_test,'R2_Test':r2_test}, index=model_names)
result_df

In [ ]:
values_estimators = [2500, 3000]
for estimator in values_estimators:
    rf = XGBRegressor(n_estimators=estimator, random_state=42 )

    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_test)
    print (f'estimator: {estimator} rmse score is : {round(np.sqrt(mean_squared_error(y_test, y_pred)),4)}, r2 score is {round(r2_score(y_test, y_pred),4)}')

In [ ]:
y_eval = model.predict(df_test)

df_sub['FloodProbability'] = y_eval

df_sub

In [ ]:
df_sub.to_csv('df_subXGB.csv', header=True, sep=',')